#Loading and inspecting data


Loading data

In [1]:
import pandas as pd

cars_df = pd.read_csv("car_prices.csv")

cars_df.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


Common description of dataset

In [3]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558837 entries, 0 to 558836
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          558837 non-null  int64  
 1   make          548536 non-null  object 
 2   model         548438 non-null  object 
 3   trim          548186 non-null  object 
 4   body          545642 non-null  object 
 5   transmission  493485 non-null  object 
 6   vin           558833 non-null  object 
 7   state         558837 non-null  object 
 8   condition     547017 non-null  float64
 9   odometer      558743 non-null  float64
 10  color         558088 non-null  object 
 11  interior      558088 non-null  object 
 12  seller        558837 non-null  object 
 13  mmr           558799 non-null  float64
 14  sellingprice  558825 non-null  float64
 15  saledate      558825 non-null  object 
dtypes: float64(4), int64(1), object(11)
memory usage: 68.2+ MB


Locating which columns have null values

In [4]:
cars_df.isnull().sum()

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
vin                 4
state               0
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64

#Filling Null values

Deleting rows according to missing values in all four columns:  Make, Model, Trim, Body

In [2]:
nan_in_all = cars_df[["make", "model", "trim", "body"]].isna().all(axis=1).sum()
nan_in_all

10301

In [3]:
rows_with_all_nan = cars_df[["make", "model", "trim", "body"]].isna().all(axis=1)

cars_df = cars_df[~rows_with_all_nan]

cars_df.shape

(548536, 16)

Deleting rows with low count of missing values

In [4]:
import pandas as pd
import numpy as np

low_missing_values_columns = ["odometer", "vin", "saledate", "sellingprice", "mmr", "body", "model", "trim"]

for column in low_missing_values_columns:
        cars_df = cars_df.dropna(subset=[column])

cars_df.shape

(545421, 16)

In [67]:
cars_df.isnull().sum()

year                0
make                0
model               0
trim                0
body                0
transmission    63152
vin                 0
state               0
condition       11141
odometer            0
color             716
interior          716
seller              0
mmr                 0
sellingprice        0
saledate            0
dtype: int64

Filling missing rows using imputation technique based on RandomForest

In [5]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



def data_for_trainning(dataset, column_as_target):

  # Print sum of all null values in dataset
  total_count = 0

  for i in range(len(cars_df.isnull().sum())):
    total_count += cars_df.isnull().sum().iloc[i]

  print(f"Total count of missing values in whole dataset: {total_count}")

  # Dropping rows with null values
  dataset.dropna(inplace=True)

  # Splitting dataset into features and target
  X = dataset.drop(column_as_target, axis=1)
  y = dataset[column_as_target]

  # Dropping unnecessary columns
  columns_to_drop = ["vin", "state", "odometer", "saledate"]
  X.drop(columns=columns_to_drop, inplace=True)

  # Taking only object columns
  object_cols = X.select_dtypes(include='object').columns

  le = LabelEncoder()

  # Apply LabelEncoder to each object column
  for col in object_cols:
      # Use LabelEncoder and fit_transform to convert categories to integers
      X[col] = le.fit_transform(X[col])


  encoders = {}
  encoders[column_as_target] = LabelEncoder()
  y = encoders[column].fit_transform(y)

  return X, y, encoders


columns_to_fill = ["transmission", "condition", "color", "interior"]

for column in columns_to_fill:

  X, y, encoders = data_for_trainning(cars_df.copy(), column)

  # Assuming X and y are your features and target variable respectively
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Instantiate an XGBoost classifier
  model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False)

  # Train the model
  model.fit(X_train, y_train)

  # Predict on the testing set
  y_pred = model.predict(X_test)

  # Calculate the accuracy
  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy for '{column}' : {accuracy * 100:.2f}%")

  cars_null_part = cars_df[cars_df[column].isna()]
  cars_filled_part = cars_df[cars_df[column].notna()]

  X_null_part = cars_null_part.drop(column, axis=1)

  columns_to_drop = ["vin", "state", "odometer", "saledate"]

  X_null_part.drop(columns=columns_to_drop, inplace=True)

  # Taking only object columns
  object_cols = X_null_part.select_dtypes(include='object').columns

  le = LabelEncoder()

  # Apply LabelEncoder to each object column
  for col in object_cols:
      # Use LabelEncoder and fit_transform to convert categories to integers
      X_null_part[col] = le.fit_transform(X_null_part[col])

  y_pred = model.predict(X_null_part)

  y_decoded = encoders[column].inverse_transform(y_pred)

  cars_null_part[column] = y_decoded

  cars_df = pd.concat([cars_filled_part, cars_null_part], ignore_index=True)

print("Filling process finished")
print(cars_df.isnull().sum())
cars_df

Total count of missing values in whole dataset: 75725
Accuracy for 'transmission' : 96.82%


<ipython-input-5-a0cfa8dff2d4>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_null_part[column] = y_decoded


Total count of missing values in whole dataset: 12573
Accuracy for 'condition' : 15.51%


<ipython-input-5-a0cfa8dff2d4>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_null_part[column] = y_decoded


Total count of missing values in whole dataset: 1432
Accuracy for 'color' : 29.66%


<ipython-input-5-a0cfa8dff2d4>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_null_part[column] = y_decoded


Total count of missing values in whole dataset: 716
Accuracy for 'interior' : 64.38%


<ipython-input-5-a0cfa8dff2d4>:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cars_null_part[column] = y_decoded


Filling process finished
year            0
make            0
model           0
trim            0
body            0
transmission    0
vin             0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
mmr             0
sellingprice    0
saledate        0
dtype: int64


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545416,2013,Mercedes-Benz,S-Class,S550 4MATIC,Sedan,automatic,wddng9eb8da504057,pa,45.0,19758.0,white,black,mercedes-benz financial services,54500.0,47700.0,Fri Dec 19 2014 08:31:00 GMT-0800 (PST)
545417,2013,Toyota,Tacoma,V6,Access Cab,automatic,5tfuu4en5dx072813,nj,1.0,1.0,blue,black,imperial auto sales inc,25800.0,6500.0,Wed Dec 17 2014 09:30:00 GMT-0800 (PST)
545418,2012,Mercedes-Benz,E-Class,E350,Convertible,automatic,wddkk5kf1cf168707,pa,47.0,13048.0,black,black,mercedes-benz financial services,37000.0,34200.0,Fri Dec 19 2014 08:31:00 GMT-0800 (PST)
545419,2014,Dodge,Grand Caravan,SXT,Minivan,automatic,2c4rdgcg8er391523,mo,1.0,24426.0,red,black,nj autofinders llc,16500.0,7300.0,Mon Dec 22 2014 11:00:00 GMT-0800 (PST)


# Formatting data set

Splitting saledate to separate columns

In [8]:
cars_df['saledate'] = pd.to_datetime(cars_df['saledate'], utc=True, errors='coerce').dt.tz_convert(None)

cars_df['sold_day_of_week'] = cars_df['saledate'].dt.day_name()  # For day name (e.g., 'Monday')
cars_df['sold_month'] = cars_df['saledate'].dt.month
cars_df['sold_year'] = cars_df['saledate'].dt.year
cars_df['sold_time'] = cars_df['saledate'].dt.strftime('%H:%M:%S')

In [9]:
cars_df

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,sold_day_of_week,sold_month,sold_year,sold_time
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 04:30:00,Tuesday,12,2014,04:30:00
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 04:30:00,Tuesday,12,2014,04:30:00
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-14 20:30:00,Wednesday,1,2015,20:30:00
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,2015-01-28 20:30:00,Wednesday,1,2015,20:30:00
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,2014-12-18 04:30:00,Thursday,12,2014,04:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545416,2013,Mercedes-Benz,S-Class,S550 4MATIC,Sedan,automatic,wddng9eb8da504057,pa,45.0,19758.0,white,black,mercedes-benz financial services,54500.0,47700.0,2014-12-19 00:31:00,Friday,12,2014,00:31:00
545417,2013,Toyota,Tacoma,V6,Access Cab,automatic,5tfuu4en5dx072813,nj,1.0,1.0,blue,black,imperial auto sales inc,25800.0,6500.0,2014-12-17 01:30:00,Wednesday,12,2014,01:30:00
545418,2012,Mercedes-Benz,E-Class,E350,Convertible,automatic,wddkk5kf1cf168707,pa,47.0,13048.0,black,black,mercedes-benz financial services,37000.0,34200.0,2014-12-19 00:31:00,Friday,12,2014,00:31:00
545419,2014,Dodge,Grand Caravan,SXT,Minivan,automatic,2c4rdgcg8er391523,mo,1.0,24426.0,red,black,nj autofinders llc,16500.0,7300.0,2014-12-22 03:00:00,Monday,12,2014,03:00:00


#Downloading cleaned df

In [10]:
cars_df.to_csv('cars_cleaned.csv', index=False)